#### Exercícios De classificação usando MLP

<img src='images/MLP.PNG'>


In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

class MLP():
    
    def __init__(self, lr, e, neurons):
        """ Construtor """
        # Taxa de aprendizado 
        self.lr = lr
        
        # tolerância
        self.e = e
        
        # Quantidade de neurônios por camadas
        self.neurons = neurons
        
        
    def sigmoid(self, valor):
        '''Calcula a sigmoid de um valor'''
        return (1/(1+math.e**(-valor)))

    def sigmoid_deriv(self, valor):
        '''Calcula a derivada da função sigmoid'''
        sig = self.sigmoid(valor)
        return sig*(1 - sig)

    def activate(self, valor):
        '''Ativa as saídas do neurônio'''
        return self.sigmoid(valor)
    
    def deriv(self, valor):
        '''Calcular a derivada da função de ativação'''
        return self.sigmoid_deriv(valor)

    def evaluate(self, target, predicted):
        '''Calcula a diferença entre o valor real e o valor predito'''
        return (target - predicted)

    def predict(self, input_data, weights):
        '''Calcula a soma ponderada das entradas pelo peso'''
        return np.dot(input_data, weights).reshape(1, -1)
    
    
    def train(self, x, d):
        ''' 
        Definir aleatoriamente os pesos, o bias e o peso do bias
        Enquanto a diferença entre m mse_anterior e o mse_atual for maior que 'e' continua o processo 
        '''
        self.w1 = np.random.random((x.shape[1]+1,self.neurons[0]))
        self.w2 = np.random.random((self.neurons[0], self.neurons[1]))
        self.w3 = np.random.random((self.neurons[1], self.neurons[2]))
        
        epoch = 0
        last_mse = np.inf
        self.total_mse = []
        self.bias = -1
        
        while True:
            mse = 0
            for xi, target in zip(x,d):
                input_value = np.insert(xi, 0, self.bias)
                i1 = self.predict(input_value, self.w1)
                y1 = self.activate(i1)
                i2 = self.predict(y1, self.w2)
                y2 = self.activate(i2)
                i3 = self.predict(y2, self.w3)
                y3 = self.activate(i3)
                current_error = self.evaluate(target, y3)
                mse+=(current_error ** 2)

                delta3 = (target - y3) * self.deriv(i3)
                self.w3 += self.lr * np.dot(y2.T, delta3)

                delta2 = np.dot(delta3, self.w3.T) * self.deriv(i2)
                self.w2 += self.lr * np.dot(y1.T, delta2)

                delta1 = np.dot(delta2, self.w2.T) * self.deriv(i1)
                self.w1 += self.lr * np.dot(input_value.reshape(1, -1).T, delta1)

            mse = sum(mse[0]) / len(x)
            
            print(f"EPOCH: {epoch} - MSE: {mse} - |mse_ant - mse|: {abs(last_mse - mse)}")
            if abs(last_mse - mse) <= self.e:
                break
            
            self.total_mse.append(mse)
            last_mse = mse
            epoch += 1
            
            
    def test(self, x):
        ''' Dado uma lista de X, submete-os à rede'''
        results = []
        for xi in x:
            input_value = np.insert(xi, 0, self.bias)
            i1 = self.predict(input_value, self.w1)
            y1 = self.activate(i1)
            i2 = self.predict(y1, self.w2)
            y2 = self.activate(i2)
            i3 = self.predict(y2, self.w3)
            y3 = self.activate(i3)
            
            results.append(y3[0])
            
        return results
    


#### Exercício 1

Use a rede Neural do Perceptron de Multicamadas para classificar os 3 tipos de flores do iris Dataset:

    0 = 'setosa'
    1 = 'versicolor'
    2 = 'virginica'

Dica: Deve haver 3 neurônios de Saída por causa das 3 saídas de neurônio. Lembre-se de organizar a saída esperada dos dados para para uma lista com índice 3.


#### Leia o Dataset e separe os dados de teste dos de treino

Pode deixar em 80% para treino

In [35]:
# Mapeando classe para vetor
def vetorizar(d):
    saidas = []
    for flor in d:
        if flor == 0:
            saidas.append(np.array([1, 0, 0]))
        elif flor == 1:
            saidas.append(np.array([0, 1, 0]))
        else:
            saidas.append(np.array([0, 0, 1]))

    return np.array(saidas)

# Mapeando vetor para classe
def classificar(vetor):
    classes = []
    for y in vetor:
        maximo = np.argmax(y)
        classes.append(maximo)
        
    return classes

In [36]:
df = pd.read_csv('iris.csv')
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [37]:
x = np.array(df.loc[:, ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']])
d = np.array(df.loc[:, 'target'])

d = vetorizar(d)

# Embaralhando os dados
shuffle = np.random.permutation(len(x))
x = x[shuffle]
d = d[shuffle]

# Separando dados de treino (80%) e teste (20%)
limite = int(len(x) * 0.8)

x_treino = x[0:limite]
x_teste = x[limite: ]

d_treino = d[0:limite]
d_teste = d[limite: ]

#### Mapeie as saídas

In [43]:
rede = MLP(lr = 0.1, e = 1e-5, neurons = [5, 4, 3])
rede.train(x = x_treino, d = d_treino)

EPOCH: 0 - MSE: 0.9976733551593995 - |mse_ant - mse|: inf
EPOCH: 1 - MSE: 0.6852689555882792 - |mse_ant - mse|: 0.3124043995711203
EPOCH: 2 - MSE: 0.6754084792488618 - |mse_ant - mse|: 0.009860476339417334
EPOCH: 3 - MSE: 0.6750355835786842 - |mse_ant - mse|: 0.0003728956701776198
EPOCH: 4 - MSE: 0.6749848618859429 - |mse_ant - mse|: 5.0721692741340085e-05
EPOCH: 5 - MSE: 0.6749580162693126 - |mse_ant - mse|: 2.684561663024443e-05
EPOCH: 6 - MSE: 0.6749293981250093 - |mse_ant - mse|: 2.861814430332732e-05
EPOCH: 7 - MSE: 0.6748946412972313 - |mse_ant - mse|: 3.475682777798195e-05
EPOCH: 8 - MSE: 0.6748516871872811 - |mse_ant - mse|: 4.295410995025506e-05
EPOCH: 9 - MSE: 0.6747982953196439 - |mse_ant - mse|: 5.3391867637198054e-05
EPOCH: 10 - MSE: 0.6747314729191999 - |mse_ant - mse|: 6.682240044397769e-05
EPOCH: 11 - MSE: 0.6746470495984115 - |mse_ant - mse|: 8.442332078839598e-05
EPOCH: 12 - MSE: 0.6745390752011192 - |mse_ant - mse|: 0.00010797439729226532
EPOCH: 13 - MSE: 0.674398872

#### Treine a Rede, teste e depois compare com o resultado de um valor de teste

(Comparar x_teste com D_teste)

In [39]:
def acuracia(real, predito):
    acertos = 0
    for i in range(len(real)):
        if real[i] == predito[i]:
            acertos+=1
    
    return acertos/len(real)

In [44]:
resultado = rede.test(x_teste)

classes_resultado = classificar(resultado)
classes_esperado = classificar(d_teste)
print("resultado")
print(classes_resultado)
print("esperado")
print(classes_esperado)
acuracia(classes_esperado, classes_resultado)

resultado
[0, 2, 2, 1, 2, 1, 2, 0, 1, 2, 0, 2, 0, 1, 1, 1, 1, 2, 1, 0, 1, 0, 0, 1, 1, 0, 0, 2, 2, 2]
esperado
[0, 2, 2, 1, 2, 1, 2, 0, 1, 2, 0, 2, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 2, 2, 2]


0.9666666666666667

#### Exercício 2

Faça o mesmo com o Dataset de Vinhos Wine Dataset

    Target = 'Wine'



#### Leia o Dataset e separe os dados de teste dos de treino

Pode deixar em 80% para treino

#### Mapeie as saídas

#### Treine a Rede, teste e depois compare com o resultado de um valor de teste

(Comparar x_teste com D_teste)